In [1]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
# This will automatically reload src/dtree.py every time you make changes and save the file
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [124]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [125]:
# combine
# fill in make
# fill in mileage, age
# fill in power and engine cap
# fill in policy-related values except depreciation rate
# calculate price based on depreciation rate
# save the values with price and evaluate the performance

In [126]:
df = pd.read_csv('./train.csv')

In [127]:
df.info()
# ['depreciation', 'omv', 'arf', 'age', 'remaining_coe', 'dereg_value', 'truck', 'suv', 'sports car', 'luxury sedan',\
#          'bus/mini bus', 'mid-sized sedan', 'stationwagon', 'hatchback', 'mpv']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16784 entries, 0 to 16783
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   listing_id         16784 non-null  int64  
 1   title              16784 non-null  object 
 2   make               14624 non-null  object 
 3   model              16784 non-null  object 
 4   description        16439 non-null  object 
 5   manufactured       16590 non-null  float64
 6   original_reg_date  318 non-null    object 
 7   reg_date           16583 non-null  object 
 8   type_of_vehicle    16784 non-null  object 
 9   category           16784 non-null  object 
 10  transmission       16784 non-null  object 
 11  curb_weight        16205 non-null  float64
 12  power              14447 non-null  float64
 13  fuel_type          3490 non-null   object 
 14  engine_cap         16731 non-null  float64
 15  no_of_owners       16608 non-null  float64
 16  depreciation       163

df_valid = df.dropna(subset = ['fuel_type'])

missingfuel_type = df.fuel_type.isnull()
dfm = df[missingfuel_type]
missingModelAge = []
for idx, row in dfm.iterrows():
    model = row['model']
    missingModelAge.append(model)

mapper = {}
for idx, row in df_valid.iterrows():
    model = row['model']
    fuel_type = row['fuel_type']
    mapper[model] = fuel_type

for idx, row in dfm.iterrows():
    model = row['model']
    if model in mapper:
        df.at[idx, 'fuel_type'] = mapper[model]

print(df['fuel_type'].isnull().sum())

In [128]:
#fill in make
dfvalid = df.dropna(subset = ['make'])

missingMake = df.make.isnull()
dfm = df[missingMake]
missingModelAge = []
for idx, row in dfm.iterrows():
    model = row['model']
    missingModelAge.append(model)

mapper = {}
for idx, row in dfvalid.iterrows():
    model = row['model']
    make = row['make']
    mapper[model] = make

for idx, row in dfm.iterrows():
    model = row['model']
    if len(mapper[model]):
        df.at[idx, 'make'] = mapper[model]

print(df['make'].isnull().sum())

0


In [129]:
# fill in mileage
df_valid = df.dropna(subset = ['mileage', 'no_of_owners'])
no_of_ownerss = df_valid['no_of_owners'].to_numpy()
caps = df_valid['mileage'].to_numpy()

k = 1000
for idx, row in df.iterrows():
    if idx % 10000 == 0:
        print(idx)

    p, e = row['no_of_owners'], row['mileage']
    
    if type(row["mileage"]) == float and pd.isna(row["mileage"]):

        diff = np.abs(no_of_ownerss - p)

        indices = np.argsort(diff)[:k]

        k_caps = caps[indices]

        avg = np.average(k_caps)

        # Set the weight value of the current row to the median
        df.at[idx, 'mileage'] = avg
print(df['mileage'].isnull().sum())

0
10000
0


In [130]:
# get age and remaining coe
d1 = df['manufactured'].isnull()
df['age'] = df['manufactured'].copy()
df.loc[d1, 'age'] = df.loc[d1, 'manufactured']
d1 = df['age'].isnull()
df.loc[d1, 'age'] = df.loc[d1, 'original_reg_date']
d1 = df['age'].isnull()
df.loc[d1, 'age'] = df.loc[d1, 'reg_date']

def month(x):
    if pd.isna(x):
        return np.nan
    mapper = {}
    all_m = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    for i in range(12):
        mapper[all_m[i]] = i + 1
    if len(str(x)) < 11:
        return (2021 - int(x)) * 12
    s = x.split('-')
    year = 2021 - int(s[-1])
    month = mapper[s[-2]]
    return year * 12 + month

def filterNeg(x):
    if x < 0:
        return 0
    return x
    
df['age'] = df['age'].map(month)

d1 = df['reg_date'].isnull()
df['remaining_coe'] = df['reg_date'].copy()
df.loc[d1, 'remaining_coe'] = df.loc[d1, 'reg_date']
d1 = df['remaining_coe'].isnull()
df.loc[d1, 'remaining_coe'] = df.loc[d1, 'original_reg_date']
d1 = df['remaining_coe'].isnull()
df.loc[d1, 'remaining_coe'] = df.loc[d1, 'manufactured']
df['remaining_coe'] = 120 - df['remaining_coe'].map(month)
df['remaining_coe'] = df['remaining_coe'].map(filterNeg)

hasLifeSpan = ~df['lifespan'].isnull()
def get_remaining(x):
    diffFrom2021 = month(x)
    res = -diffFrom2021
    if res < 0:
        res = 0
    return res ##check 2021
df.loc[hasLifeSpan, 'remaining_coe'] = df.loc[hasLifeSpan, 'lifespan'].map(get_remaining)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16784 entries, 0 to 16783
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   listing_id         16784 non-null  int64  
 1   title              16784 non-null  object 
 2   make               16784 non-null  object 
 3   model              16784 non-null  object 
 4   description        16439 non-null  object 
 5   manufactured       16590 non-null  float64
 6   original_reg_date  318 non-null    object 
 7   reg_date           16583 non-null  object 
 8   type_of_vehicle    16784 non-null  object 
 9   category           16784 non-null  object 
 10  transmission       16784 non-null  object 
 11  curb_weight        16205 non-null  float64
 12  power              14447 non-null  float64
 13  fuel_type          3490 non-null   object 
 14  engine_cap         16731 non-null  float64
 15  no_of_owners       16608 non-null  float64
 16  depreciation       163

In [131]:
df = df.dropna(subset=['omv', 'depreciation', 'dereg_value'])

In [132]:
# age,power, enginecap
df.loc[df.age < 0, 'age'] = 0
df.loc[df.age > 20000, 'age'] = 0

In [133]:
##train
print(df[df.engine_cap == 0])
df.at[8199, 'engine_cap'] = np.nan

      listing_id                                              title  \
8199     1005670  Mercedes-Benz E-Class E300e Plug-in Hybrid AMG...   

               make model                                        description  \
8199  mercedes-benz   amg  great deal! grab it! new facelifted 2021 with ...   

      manufactured original_reg_date     reg_date type_of_vehicle  \
8199        2021.0               NaN  16-jun-2021    luxury sedan   

                                               category  ...      arf  \
8199  parf car, almost new car, low mileage car, hyb...  ...  66486.0   

      opc_scheme  lifespan   eco_category  \
8199         NaN       NaN  uncategorized   

                                               features  \
8199  2.0l m274 powerplant with eq boost producing 3...   

                                            accessories  indicative_price  \
8199  amg kits and wheels, memory seat, ambient ligh...               NaN   

         price  age  remaining_coe  
8199  24

In [134]:
df_valid = df.dropna(subset = ['power'])
ps = df_valid['power'].to_numpy()

missingPs = df.power.isnull()
dfm = df[missingPs]
missingPower = []
for idx, row in dfm.iterrows():
    model = row['model']
    missingPower.append(model)

mapper = defaultdict(list)
for idx, row in df_valid.iterrows():
    model = row['model']
    power = row['power']
    mapper[model].append(power)

for idx, row in dfm.iterrows():
    model = row['model']
    if len(mapper[model]):
        df.at[idx, 'power'] = np.average(mapper[model])

print(df['power'].isnull().sum())

1469


In [135]:
df_valid = df.dropna(subset = ['engine_cap'])
ps = df_valid['engine_cap'].to_numpy()

missingPs = df.power.isnull()
dfm = df[missingPs]
missingPower = []
for idx, row in dfm.iterrows():
    model = row['model']
    missingPower.append(model)

mapper = defaultdict(list)
for idx, row in df_valid.iterrows():
    model = row['model']
    power = row['engine_cap']
    mapper[model].append(power)

for idx, row in dfm.iterrows():
    model = row['model']
    if len(mapper[model]):
        df.at[idx, 'engine_cap'] = np.average(mapper[model])

print(df['engine_cap'].isnull().sum())

49


In [136]:
df_valid = df.dropna(subset = ['power', 'engine_cap'])
powers = df_valid['power'].to_numpy()
caps = df_valid['engine_cap'].to_numpy()

k = 100

# Loop over each data point in our dataset
for idx, row in df.iterrows():
    p, e = row['power'], row['engine_cap']
    
    if type(row["power"]) == float and pd.isna(row["power"]):
        diff = np.abs(caps - e)

        indices = np.argsort(diff)[:k]

        k_powers = powers[indices]

        avg = np.average(k_powers)

        # Set the weight value of the current row to the median
        df.at[idx, 'power'] = avg

In [137]:
df_valid = df.dropna(subset = ['power', 'engine_cap'])
powers = df_valid['power'].to_numpy()
caps = df_valid['engine_cap'].to_numpy()

k = 50
for idx, row in df.iterrows():
    if idx % 10000 == 0:
        print(idx)

    p, e = row['power'], row['engine_cap']
    
    if type(row["engine_cap"]) == float and pd.isna(row["engine_cap"]):

        diff = np.abs(powers - p)

        indices = np.argsort(diff)[:k]

        k_caps = caps[indices]

        avg = np.average(k_caps)

        # Set the weight value of the current row to the median
        df.at[idx, 'engine_cap'] = avg

0
10000


df_ = df.copy()
df_['efficiency'] = df_['power']
for i in range(len(df_)):
    x = df_.iloc[i]
    df_['efficiency'][i] = x['power'] / x['engine_cap']

plt.boxplot(df_[df_.fuel_type == 'diesel']['efficiency'])# / diesel['engine_cap'])
plt.show()

In [23]:
# plt.boxplot(df['power'])# / diesel['engine_cap'])
# plt.show()

In [24]:
# plt.boxplot(df['engine_cap'])# / diesel['engine_cap'])
# plt.show()

plt.boxplot(df_[df_.fuel_type == 'petrol']['efficiency'])# / diesel['engine_cap']) ##outlier
plt.show()

# fill omv, get data with missing omv and dereg_value, fill in the omv
is_ = df[df.omv.isnull() & df.dereg_value.isnull()].index.tolist()
filled_omvs = []
for idx in is_:
    ag = df.at[idx, 'age']
    price = df.at[idx, 'price']
    mo = reg.predict(np.array([[ag, price]]))
    filled_omvs.append(mo)
for i, idx in enumerate(is_):
    df.at[idx, 'omv'] = filled_omvs[i]

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16749 entries, 0 to 16783
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   listing_id         16749 non-null  int64  
 1   title              16749 non-null  object 
 2   make               16749 non-null  object 
 3   model              16749 non-null  object 
 4   description        16404 non-null  object 
 5   manufactured       16558 non-null  float64
 6   original_reg_date  312 non-null    object 
 7   reg_date           16550 non-null  object 
 8   type_of_vehicle    16749 non-null  object 
 9   category           16749 non-null  object 
 10  transmission       16749 non-null  object 
 11  curb_weight        16205 non-null  float64
 12  power              16749 non-null  float64
 13  fuel_type          3489 non-null   object 
 14  engine_cap         16749 non-null  float64
 15  no_of_owners       16575 non-null  float64
 16  depreciation       163

In [138]:
# coe: fill in missing value for coe according to: age and power
fill_arf = df[df.arf.isnull()]
for idx, row in fill_arf.iterrows():
    omv = row['omv']
    if omv < 20000:
        df.at[idx, 'arf'] = omv
    elif omv >= 20000 and omv <= 50000:
        df.at[idx, 'arf'] = (omv - 20000) * 1.40 + 20000
    else:
        df.at[idx, 'arf'] =  (omv - 50000) * 1.80 + 62000
print(len(df[df.arf.isnull()]))
# df = df[~(df.arf.isnull() & df.dereg_value.isnull())] # remove values whose dereg_value cannot be calculated

0


In [104]:
# for date missing both coe and dereg_value, record it's month engine and power
coe_to_search = []
to_search = df[df.coe.isnull() & df.dereg_value.isnull()]

for idx, row in to_search.iterrows():
    # category -> A / B
    power = row["power"]
    engine_cap = row["engine_cap"]
    cat = 2
    month = row["age"]
    if engine_cap <= 1600 and power <= 97:
        cat = 1
    coe_to_search.append((cat, month))

coe_to_search = list(set(coe_to_search))
print(len(coe_to_search))

0


In [55]:
print(len(df[df.coe.isnull()]))

10


In [139]:
df = df.dropna(subset = ['coe'])

In [57]:
print(len(df[df.coe.isnull()]))

0


In [140]:
df.info()
df.to_csv('trian_clean.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15424 entries, 0 to 16783
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   listing_id         15424 non-null  int64  
 1   title              15424 non-null  object 
 2   make               15424 non-null  object 
 3   model              15424 non-null  object 
 4   description        15099 non-null  object 
 5   manufactured       15420 non-null  float64
 6   original_reg_date  21 non-null     object 
 7   reg_date           15424 non-null  object 
 8   type_of_vehicle    15424 non-null  object 
 9   category           15424 non-null  object 
 10  transmission       15424 non-null  object 
 11  curb_weight        15100 non-null  float64
 12  power              15424 non-null  float64
 13  fuel_type          3283 non-null   object 
 14  engine_cap         15424 non-null  float64
 15  no_of_owners       15419 non-null  float64
 16  depreciation       154

In [141]:
prices = df['price']

for idx in range(len(df)):
    row = df.iloc[idx]
    depreciation = row['depreciation']
    age = row["age"]
    coe = row["coe"]
    omv = row["omv"]
    arf = row["arf"]
    remaining_coe = row['remaining_coe']

    min_parf = min(0.55 * omv, arf)
    if age <= 96:
        min_parf = 0.5 * arf ####
    elif age > 96 and age <= (2021 - 2008) * 12:
        min_parf = min(omv, arf)
    
    # print(idx, len(prices))
    new_coe = (prices.iloc[idx] - 0.5 * min_parf) / depreciation
    if new_coe <= 120 and new_coe >= 0 and age >= (2021 - 2002) * 12:
        df.iloc[idx, 31] = new_coe


In [118]:
df_test = pd.read_csv('./test/without_null_dep.csv')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       5000 non-null   int64  
 1   mileage          5000 non-null   float64
 2   depreciation     5000 non-null   float64
 3   omv              5000 non-null   float64
 4   age              5000 non-null   int64  
 5   remaining_coe    5000 non-null   int64  
 6   dereg_value      5000 non-null   float64
 7   truck            5000 non-null   int64  
 8   suv              5000 non-null   int64  
 9   sports car       5000 non-null   int64  
 10  luxury sedan     5000 non-null   int64  
 11  bus/mini bus     5000 non-null   int64  
 12  mid-sized sedan  5000 non-null   int64  
 13  stationwagon     5000 non-null   int64  
 14  hatchback        5000 non-null   int64  
 15  mpv              5000 non-null   int64  
dtypes: float64(4), int64(12)
memory usage: 625.1 KB


In [111]:
df['fuel_type'] = df['fuel_type'].fillna('petrol')
cars_dummies = pd.get_dummies(df['type_of_vehicle'])
train = pd.concat([df, cars_dummies], axis=1)
cars_dummies = pd.get_dummies(df['fuel_type'])
train = pd.concat([train, cars_dummies], axis=1)

train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15434 entries, 0 to 16783
Data columns (total 47 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   listing_id         15434 non-null  int64  
 1   title              15434 non-null  object 
 2   make               15434 non-null  object 
 3   model              15434 non-null  object 
 4   description        15109 non-null  object 
 5   manufactured       15430 non-null  float64
 6   original_reg_date  21 non-null     object 
 7   reg_date           15434 non-null  object 
 8   type_of_vehicle    15434 non-null  object 
 9   category           15434 non-null  object 
 10  transmission       15434 non-null  object 
 11  curb_weight        15110 non-null  float64
 12  power              15434 non-null  float64
 13  fuel_type          15434 non-null  object 
 14  engine_cap         15434 non-null  float64
 15  no_of_owners       15429 non-null  float64
 16  depreciation       154

In [142]:
useful = ['mileage', 'depreciation', 'omv', 'age', 'remaining_coe', 'dereg_value', 'truck', 'suv', 'sports car', 'luxury sedan',\
         'bus/mini bus', 'mid-sized sedan', 'stationwagon', 'hatchback', 'mpv']#, 'coe'] #, engine_cap, power
train = train[useful] ##rank usefulness, arf
train.info()

prices = df['price']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15434 entries, 0 to 16783
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   mileage          15434 non-null  float64
 1   depreciation     15434 non-null  float64
 2   omv              15434 non-null  float64
 3   age              15434 non-null  int64  
 4   remaining_coe    15434 non-null  int64  
 5   dereg_value      15434 non-null  float64
 6   truck            15434 non-null  uint8  
 7   suv              15434 non-null  uint8  
 8   sports car       15434 non-null  uint8  
 9   luxury sedan     15434 non-null  uint8  
 10  bus/mini bus     15434 non-null  uint8  
 11  mid-sized sedan  15434 non-null  uint8  
 12  stationwagon     15434 non-null  uint8  
 13  hatchback        15434 non-null  uint8  
 14  mpv              15434 non-null  uint8  
dtypes: float64(4), int64(2), uint8(9)
memory usage: 1.5 MB


In [143]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, prices, 
                                                    train_size=0.7,
                                                    test_size = 0.3, random_state=100)
model = RandomForestRegressor(n_estimators = 400, max_features = 8, random_state=0)
# model = RandomForestRegressor(n_estimators = 300)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse=mean_squared_error(y_test, y_pred)
print(mse)

ValueError: Found input variables with inconsistent numbers of samples: [15434, 15424]

In [116]:
model = RandomForestRegressor(n_estimators = 450, max_features = 8, random_state=0)
# # model = RandomForestRegressor(n_estimators = 300)
model.fit(train, prices)
# model.fit(X, y)

RandomForestRegressor(max_features=8, n_estimators=450, random_state=0)

In [121]:
cols = ['mileage', 'depreciation', 'omv', 'age', 'remaining_coe', 'dereg_value', 'truck', 'suv', 'sports car', 'luxury sedan',\
         'bus/mini bus', 'mid-sized sedan', 'stationwagon', 'hatchback', 'mpv']
df_test = df_test[cols]

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   mileage          5000 non-null   float64
 1   depreciation     5000 non-null   float64
 2   omv              5000 non-null   float64
 3   age              5000 non-null   int64  
 4   remaining_coe    5000 non-null   int64  
 5   dereg_value      5000 non-null   float64
 6   truck            5000 non-null   int64  
 7   suv              5000 non-null   int64  
 8   sports car       5000 non-null   int64  
 9   luxury sedan     5000 non-null   int64  
 10  bus/mini bus     5000 non-null   int64  
 11  mid-sized sedan  5000 non-null   int64  
 12  stationwagon     5000 non-null   int64  
 13  hatchback        5000 non-null   int64  
 14  mpv              5000 non-null   int64  
dtypes: float64(4), int64(11)
memory usage: 586.1 KB


In [122]:
y_pred = model.predict(df_test)
y_pred = pd.DataFrame(y_pred)

y_pred = y_pred.rename(columns={0: 'Predicted'})
# y_pred = y_pred.dropna()
y_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Predicted  5000 non-null   float64
dtypes: float64(1)
memory usage: 39.2 KB


In [123]:
y_pred.to_csv('predictions_miss_2x.csv')

In [45]:
new_preds = y_pred['Predicted'].tolist()

In [46]:
cur_best = pd.read_csv('highest.csv')
cur_pred = cur_best['Predicted'].tolist()

In [50]:
print(len(new_preds))
dropped = set(L)
print(len(dropped))

l1 = len(omvL)
l2 = len(depL)
print(l1, l2)

4868
141
10 132


In [62]:
combined_preds = []
j = 0
for i in range(5000):
    if i in depL:
        old = cur_pred[i]
        combined_preds.append(old)
    else:
        new = new_preds[j]
        combined_preds.append(new)
        j += 1

# print(combined_preds[-3:])
# print(new_preds[-3:])
for o in omvL:
    combined_preds[o] = cur_pred[o]

In [64]:
sr = pd.DataFrame(combined_preds)
sr = sr.rename(columns={0: 'Predicted'})
sr.info()
sr.to_csv('predictions.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Predicted  5000 non-null   float64
dtypes: float64(1)
memory usage: 39.2 KB
